# Tripadvisor top 100 London Restaurants

I have always been a foodie and have enjoyed many great cuisines and restaurants in London. London is a melting pot of cultures with a large selection of the best food from around the world. 

According to the ONS, as of 2018, there are 15,550 restaurants in London. What does it take for a restaurant to be the top100 on a popular restaurant recommender website, such as Tripadvisor

This project is to scrape data from the top 100 restaurants on tripadvisor and to find out features that makes the restaurant a top 100 reviewed location. 

A data visualization is done on Tableau to illustrate the point more clearly


This part is to scrape the rank,average rating,name,address,borough,price_range,cuisine of each of the top 100 restaurants on Tripadvisor. 

In [1]:
# initialize the work environment
import csv
import requests
import time 
from selenium import webdriver
import pandas as pd
import numpy as np
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [3]:
# scrape data using selenium 
# creating path on selenium 
website = 'https://www.tripadvisor.com/Restaurants-g186338-London_England'
path = 'C:/Users/Kelvin/chromedriver_win32/chromedriver'
driver =  webdriver.Chrome(path)
driver.get(website)

In [4]:
# click button on website to accept value privacy by building the xpath 
time.sleep(20)
try: 
    privacy_trust_button =  driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]')
    privacy_trust_button.click()
except NoSuchElementException: 
    pass

In [5]:
# find and save restaurant features (name, location, price range, cuisine, average ratings,reviews)
restaurant_url = []
# extracting data by element xpath
for i in range(0,3):
    time.sleep(5)
    urls = driver.find_elements_by_xpath("//a[@class='bHGqj Cj b']")
    for url in urls:
        restaurant_url.append(url.get_attribute('href'))
    #next page
    urls = []
    time.sleep(5)
    element = driver.find_element_by_xpath("//*[@class='nav next rndBtn ui_button primary taLnk']")
    driver.execute_script("arguments[0].click();", element)


In [ ]:
# initialize csv to save data 
path1 = "/Users/Kelvin/Data Science Project/EDA and Data Cleaning Project/reviews.csv"
csvf = open(path1, 'a', encoding="utf-8")
csvWriter = csv.writer(csvf)

# initialize the features to be saved into dataframe
rank=[]
name=[]
address=[]
borough=[]
price_range=[]
cuisine=[]
average_rating=[]
#scrape each feature using selenium via finding element through xpath
for urls in restaurant_url:
    driver =  webdriver.Chrome(path)
    driver.get(urls)
    wait = WebDriverWait(driver, 20)
    time.sleep(10)
    try: 
        privacy_trust_button =  driver.find_element_by_xpath('//button[@id="onetrust-accept-btn-handler"]')
        privacy_trust_button.click()
    except NoSuchElementException: 
        pass   
    time.sleep(20)
    
    try:
        rank1=driver.find_element_by_xpath("//div[@class='fYCpi']/child::b/child::span").text
        rank.append(rank1)
    except NoSuchElementException: 
        rank2 = np.nan
        rank.append(rank2)

    try:    
        name1 = driver.find_element_by_xpath("//div[@h1]" and "//h1[@class='fHibz']").text
        name.append(name1)
    except NoSuchElementException: 
        name2 = np.nan
        name.append(name2)
    time.sleep(2)

    try: 
        address1 = driver.find_element_by_xpath("//span[@class='brMTW']/ancestor::div[@class='cSPba bKBJS Me']").text
        address.append(address1)
    except NoSuchElementException: 
        address2 = np.nan
        address.append(address2)
    try:    
        borough1 = driver.find_element_by_xpath("//span[@class='ui_icon neighborhoods bPFFU QKJnF']/following-sibling::span/child::div[1]").text
        borough.append(borough1)
    except NoSuchElementException: 
        borough2 = np.nan
        borough.append(borough2)
        
    try: 
        average_rating1 = driver.find_element_by_xpath("//span[@class='fdsdx']").text
        average_rating.append(average_rating1)
    except NoSuchElementException: 
        average_rating2 = np.nan
        average_rating.append(average_rating2)
    
    try: 
        price_range1 = driver.find_element_by_xpath("(//a[@class='drUyy'])[1]")
        price_range1.location_once_scrolled_into_view
        price_range.append(price_range1.text)
            
    except NoSuchElementException: 
        price_range2 = np.nan
        price_range.append(price_range2)

    try: 
        cuisine1 = driver.find_element_by_xpath("(//a[@class='drUyy'])[2]")
        cuisine1.location_once_scrolled_into_view
        if len(cuisine1.text.split(','))>1:
            cuisine2=cuisine1.text.split(',')[1]
            cuisine.append(cuisine2)
        else:
            cuisine2=cuisine1.text
            cuisine.append(cuisine2)
    except NoSuchElementException: 
        cuisine3 = np.nan
        cuisine.append(cuisine3)      
    
    # scraping all the reviews for the top 100 restaurants 
    review_container = driver.find_elements_by_xpath(".//div[@class='review-container']")
    time.sleep(2)
    for r in range(len(review_container)):    
        naming = driver.find_element_by_xpath(("//div[@h1]" and "//h1[@class='fHibz']") or "//h1[@class='header heading masthead masthead_h1 ']").text
        time.sleep(2)
        reviews = review_container[r].find_element_by_xpath(".//p[@class='partial_entry']").text

        review_dates = review_container[r].find_element_by_xpath("//*[@class='ratingDate']").get_attribute("title")

        ratings = review_container[r].find_element_by_xpath(".//span[contains(@class, 'ui_bubble_rating bubble_')]").get_attribute("class").split("_")[3]              

        csvWriter.writerow([naming,review_dates, ratings, reviews])   
    driver.close()


In [ ]:
# saving all the data into the dataframe 
d={'rank':rank,'name':name,'borough':borough,'price range':price_range,'cuisine':cuisine,'average rating':average_rating,'address':address} 
df=pd.DataFrame.from_dict(d)
print(df)
df.to_csv('restaurant_info.csv')

In [ ]:
with open("reviews.csv", 'w') as file:
    headerlist = ['restaurant name','review date','rating','review' ]
    writing = csv.DictWriter(file, delimiter=',', 
                        fieldnames=headerlist)
    writing.writeheader()

with open("restaurant_info.csv", 'w') as file:
    headerlist1 = ['rank', 'name','borough','price_range','cuisine','average rating']
    writing1 = csv.DictWriter(file, delimiter=',', 
                        fieldnames=headerlist1)
    writing1.writeheader()

In [ ]:
# compile all strings into a dataframe

In [ ]:
# clean data 

In [ ]:
# EDA preprocess data for ml models 

In [ ]:
# sentiment analysis of reviews + combine population data in borough to predict correlation between features (i.e. price range vs local salary)
# find out best area to open up certain restaurants (i.e. Earls court - Chinese restaurant)

In [ ]:
# ML model reference: https://medium.com/@zhiwei_zhang/final-blog-642fb9c7e781     OR   http://www.apjis.or.kr/pdf/02_%EC%9A%B0%EC%A2%85%EC%9A%B1(35-49).pdf
# Sentiment analysis reference: https://monkeylearn.com/blog/sentiment-analysis-tripadvisor/


